# Command Registry

In [ ]:
%pip install -q ipylab

In [ ]:
import ipylab

app = await ipylab.App().ready()
await app.commands.ready()

In [ ]:
app.version

## List all commands

In [ ]:
out = ipylab.SimpleOutput(layout={"height": "200px", "overflow": "auto"}).add_class("ipylab-ResizeBox")
out.push(app.commands.all_commands)

## Create a new console

In [ ]:
await app.commands.execute(
    "console:create",
    {
        "insertMode": "split-right",
        "kernelPreference": {
            "shutdownOnClose": True,
        },
    },
)

## Change the theme

In [ ]:
await app.commands.execute("apputils:change-theme", {"theme": "JupyterLab Dark"})

In [ ]:
await app.commands.execute("apputils:change-theme", {"theme": "JupyterLab Light"})

## Create a new terminal

In [ ]:
await app.commands.execute("terminal:create-new")

## Add your own command

Let's create a nice plot with `bqlot` and generate some random data. 

See https://github.com/bqplot/bqplot/blob/master/examples/Advanced%20Plotting/Animations.ipynb for more details.

Note: This requires bqplot and numpy to be installed, which may require Jupyterlab to be restarted if it hasn't already been installed.

In [ ]:
import numpy as np
from bqplot import Axis, Bars, Figure, LinearScale, Lines

import ipylab

app = ipylab.App()

In [ ]:
xs = LinearScale()
ys1 = LinearScale()
ys2 = LinearScale()

x = np.arange(20)
y = np.cumsum(np.random.randn(20))
y1 = np.random.rand(20)

line = Lines(x=x, y=y, scales={"x": xs, "y": ys1}, colors=["magenta"], marker="square")
bar = Bars(x=x, y=y1, scales={"x": xs, "y": ys2}, colorpadding=0.2, colors=["steelblue"])

xax = Axis(scale=xs, label="x", grid_lines="solid")
yax1 = Axis(scale=ys1, orientation="vertical", tick_format="0.1f", label="y", grid_lines="solid")
yax2 = Axis(scale=ys2, orientation="vertical", side="right", tick_format="0.0%", label="y1", grid_lines="none")

fig = Figure(marks=[bar, line], axes=[xax, yax1, yax2], animation_duration=1000)
panel = ipylab.Panel([fig])
await panel.add_to_shell(mode=ipylab.InsertMode.split_right)

We now define a function to update the data.

In [ ]:
def update_data():
    line.y = np.cumsum(np.random.randn(20))
    bar.y = np.random.rand(20)

In [ ]:
update_data()

This function will now be called when the JupyterLab command is executed.

> Commands can also custom [icons](./icons.ipynb) in place of `icon_class`.

In [ ]:
cmd = await app.commands.add_command(
    "update_data", execute=update_data, label="Update Data", icon_class="jp-PythonIcon"
)

In [ ]:
await app.commands.execute("update_data")

Execute it!

The plot should have updated with new values.

Also the list of commands gets updated with the newly added command:

The command `connection_id` ends with the `name`. The command id in the registry is the 'connection_id'. We use the 'connection_id' to avoid name clashes between multiple kernels.

In [ ]:
command_id = str(cmd)

In [ ]:
assert command_id in app.commands.all_commands  # noqa: S101

That's great, but the command doesn't visually show up in the palette yet. So let's add it!

## Add the command to the palette

In [ ]:
await app.command_pallet.add(command=cmd, category="Python Commands")

Open the command palette `CTRL + SHIFT + C` and the command should show now be available as `Update data`.

## Add the command to the context menu (Right click)

In [ ]:
cm = await app.context_menu.add_item(command=cmd)

Right click on the plot to open the context menu.

## Add a key binding to the command

Add a 'hot key' for any Ipylab widget in the main area.

In [ ]:
# Use uppercase
kb = await cmd.add_key_binding(["U"])

In [ ]:
kb in cmd.key_bindings

Click on the plot and press `u`

Note: By default will block the key 'u' for all widgets. Pass `prevent_default=False` to allow the keybinding to propagate.   

### Selector

As with the context menu, the key bindings are scoped according to the `selector`.  To make the keybinding relevant Jupyterlab wide, the selector `.jp-ThemedContainer` can be used.

In [ ]:
await cmd.add_key_binding(["Ctrl 1"], selector=".jp-ThemedContainer")

Press `Ctrl 1` to update the plot.

## Remove a command

To remove a command that was previously added:

In [ ]:
cmd.close()

## Other command registries

It is also possible to create command registries separate to the Jupyterlab command registry.

See [menu->limiting scope](menu.ipynb#Limiting-scope) for an example using one.

In [ ]:
panel.close()